## Seleccionamos la Colección y definimos el rango en el que queremos trabajar

In [67]:
from collections import defaultdict
from pymongo import MongoClient
import time
import operator
import datetime
import re

client = MongoClient()
db = client.twitter
te = db['prueba2']
inicio =  datetime.datetime(2017, 4, 14)
fin = datetime.datetime(2017,4,15)
limit = 100

## Creamos un diccionario por URLS y metemos primera fecha

In [71]:
dw = defaultdict(lambda: [0, datetime.datetime.now()])
for a in te.find({'$and': [{'created_at': {"$lt" : fin}} ,{'created_at': {"$gt" : inicio}} ]}):
    if len(a['entities']['urls']) >0:
        for b in a['entities']['urls']:
            if dw[b['expanded_url']]:
                dw[b['expanded_url']][0] += 1
                if(a['created_at'] < dw[b['expanded_url']][1]):
                    dw[b['expanded_url']][1]= a['created_at']
    if 'retweeted_status' in a and len(a['retweeted_status']['entities']['urls']) >0:      
        for b in a['retweeted_status']['entities']['urls']:
                dw[b['expanded_url']][0] +=  1
                if(a['created_at'] < dw[b['expanded_url']][1]):
                    dw[b['expanded_url']][1]= a['created_at']

## Creamos lista ordenada a partir del diccionario 

In [94]:
#print dw
lista = map (lambda (x,y): [x,y[0],y[1]],dw.items())
trending = sorted(lista, key=operator.itemgetter(1), reverse=True)
#print trending
i = 1
#trending = []
urls = [] 
medias = ['elpais', 'elmundo', 'eleconomista', 'elmundotoday', 'marca com', '20minutos', 'meneame', 'quora']
for a in trending:
    if any(word in str(a[0]) for word in medias):
        urls.append(a)
        i = i+1
    if i > limit:
        break

## Analizamos la info de las urls extraidas

In [62]:
import urllib2
from bs4 import BeautifulSoup

def get_data_pais(name):
    opener = urllib2.build_opener()
    opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
    webpage = opener.open(name).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    title = soup.find("meta",  property="og:title")
    title2 = soup.find("meta",  {"name":"title"})
    title3 = soup.find("title")
    url = soup.find("meta",  property="og:url")
    desc = soup.find("meta",  property="og:description")
    desc2 = soup.find("meta",  {"name":"description"})
    img = soup.find("meta",  property="og:image")
    d = {}
    if title3 :
            title3= str(title3).replace("<title>", "").replace("</title>", "")
    d['title'] = title["content"] if title else (title2["content"] if title2 else title3)
    d['url'] = url["content"] if url else name
    d["desc"] = desc["content"] if desc else (desc2["content"] if desc2 else None)
    d["img"] = img["content"] if img else None
    
    if not d["img"]:
        imgs = soup.findAll("div", {"class":"article-image"})
        for img in imgs:
            soup2 = BeautifulSoup(str(img), 'html.parser')
            i = soup2.findAll("img")
            if i:
                i = str(i)[str(i).index('src="')+5:]
                i = i[:i.index('"')]
                d["img"] = i
    
    
    return d
  
lurls = []
#urls = [[r'http://m.20minutos.es/deportes/noticia/der-spiegel-cristiano-ronaldo-pago-euros-evitar-juicio-presunta-violacion-3012042/0/',4]]



for url in urls:
    d = get_data_pais(url[0])
    d['rank'] = url[1]
    lurls.append(d)
    



In [63]:

for d in lurls:
    if not d['title']:
        print d

## Volcamos a json

In [65]:
import json
with open('tweevy', 'w') as f:
    f.write(json.dumps(lurls))